In [1]:
%%capture
try:
    import transformers4rec
except:
    print("Install packages\n\n")
    !pip install transformers4rec[pytorch,nvtabular]
    !pip install -U nvtabular==1.3.3
    !pip install beartype
    !pip install -U pytorch_lightning

In [ ]:
import os
import ast
import json
import glob
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import nvtabular as nvt
import matplotlib.pyplot as plt
import gc
import cudf
import joblib

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn import preprocessing

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

from nvtabular.ops import *
from merlin.schema.tags import Tags
from merlin_standard_lib import Schema
# from transformers4rec import torch as tr
# from transformers4rec.torch import Trainer
# from transformers4rec.torch.ranking_metric import RecallAt
from nvtabular.loader.torch import TorchAsyncItr, DLDataLoader
from transformers4rec.config.trainer import T4RecTrainingArguments

In [4]:
TRAIN_PATH = '/kaggle/input/otto-features/output/part_0.parquet'
train_w1 = cudf.read_parquet(TRAIN_PATH)

In [5]:
train_w1

,session,clicks_Monday-list,clicks_Tuesday-list,clicks_Wednesday-list,clicks_Thursday-list,clicks_Friday-list,clicks_Saturday-list,clicks_Sunday-list,carts_Monday-list,carts_Tuesday-list,...,carts_Sunday-list,order_Monday-list,order_Tuesday-list,order_Wednesday-list,order_Thursday-list,order_Friday-list,order_Saturday-list,order_Sunday-list,aid-list,type-list
0,0,"[0, 0, 2548, 26087, 285103, 567009, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[184150, 301725, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 43018, 1821, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 14092, 1821, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[184150, 301725, 2548, 26087, 285103, 567009, ...","[0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, ..."
1,1,[0],[0],[0],[0],[0],[0],[0],[0],[0],...,[618079],[0],[0],[0],[0],[0],[0],[0],[618079],[1]
2,2,[0],[0],[0],[0],[0],[0],[16267],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[16267],[0]
3,3,"[0, 0, 5808, 5808, 0, 22328, 5808, 94679, 1733...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 22328, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 5808, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[22328, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[22328, 22328, 5808, 5808, 5808, 22328, 5808, ...","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[143678, 56718]","[0, 0]","[0, 0]",...,"[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[143678, 56718]","[0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4441137,5348699,[0],[0],[0],[0],[0],[0],[6],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[6],[0]
4441138,5348703,[0],[0],[0],[0],[0],[0],[161],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[161],[0]
4441139,5348707,[0],[0],[0],[0],[0],[0],[907453],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[907453],[0]
4441140,5348718,[0],[0],[0],[0],[0],[0],[24748],[0],[0],...,[0],[0],[0],[0],[0],[0],[0],[0],[24748],[0]
